### High-dimensional synthetic data

- C: Vector of {c_dim} {dice_size}-sided dice rolls.
- A: Flip 1 + {dice_size} - median(C) coins. A is 1 if at least one flip comes up heads.
- Y: Flip f(C) + A coins and write down the number of heads.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import time

In [2]:
def get_smf_model_a_param(ols, df):
    """
    Fit a model with statsmodels
    Return the parameter corresponding to the treatment
    """
    return smf.ols(ols, data=df).fit().params['a']

In [12]:
def raise_c_to_power(c, power):
    arr = []
    for p in range(1, 1 + power):
        arr.append(np.power(c, p))
    return np.concatenate(arr, axis=1)

raise_c_to_power(np.array([[1, 2, 3]]), 3)

array([[ 1,  2,  3,  1,  4,  9,  1,  8, 27]])

In [4]:
def observed(n=100, c_dim=6, dice_size=2, power=1, base_coef=1.1):
    """
    The observed data distribution
      C: roll {c_dim} {dice_size}-sided dice and record the results
      A: flip `1 + dice_size - np.median(C)` fair coins
          and record 1 if at least one flip lands heads
      
      Y: flip `C + A` fair coins, and record the number of heads
    """

    c_coefs = np.array([(-1) ** i * j * base_coef ** i
                        for j in range(1, 1 + power)
                        for i in range(1, 1 + c_dim)])

    # what's a lower bound on `raise_c_to_power(c, power) @ c_coefs`? Subtract that off.
    all_ones = np.ones([1, c_dim])
    low_roll = raise_c_to_power(all_ones, power) * c_coefs
    high_roll = raise_c_to_power(all_ones * dice_size, power) * c_coefs
    worst_roll = np.min(np.concatenate([low_roll, high_roll], axis=0), axis=0)
    y_min_dice = np.sum(worst_roll)
          
    c = np.random.randint(1, 1 + dice_size, (n, c_dim))
    c_median = np.median(c, axis=1).astype(int)
    a = np.random.binomial(n=1 + dice_size - c_median, p=0.5, size=n)
    a = (a > 0).astype(np.int32)

    full_c = raise_c_to_power(c, power)
    y_n_dice = 1 + np.ceil(-y_min_dice + a + full_c @ c_coefs).astype(int)
    y = np.random.binomial(n=y_n_dice, p=0.5)

    columns = {"a": a, "y": y}
    # c_col_names = [f"c{i}_{j}" for i in range(1, 1 + c_dim) for j in range(1, 1 + power)]
    # c_cols = [col.reshape(-1) for col in np.array_split(c, c_dim * power, axis=1)]
    c_col_names = [f"c{i}" for i in range(1, 1 + c_dim)]
    c_cols = [col.reshape(-1) for col in np.array_split(c, c_dim, axis=1)]
    columns.update(dict(zip(c_col_names, c_cols)))
    df = pd.DataFrame(data=columns)

    return df

In [5]:
def experiment(estimator="ols", n=100, c_dim=6,
               repeats=1, power=1, dice_size=2,
               ground_truth=None, prec=(3,0)):

    # c_col_names = [f"c{i}_{j}" for i in range(1, 1 + c_dim) for j in range(1, 1 + power)]
    c_col_names = [f"c{i}" for i in range(1, 1 + c_dim)]
    results = []
    np.random.seed(42)
    for i in range(repeats):
        df = observed(n=n, c_dim=c_dim, power=power, dice_size=dice_size)

        if estimator == "ols":
            ols = "y ~ a + " + " + ".join(c_col_names)
            results.append(get_smf_model_a_param(ols, df))

        elif estimator == "count":
            total = 0
            denominator = 0
            unique_c, counts = np.unique(
                df[c_col_names], axis=0, return_counts=True)

            for uniq, count in zip(unique_c, counts):
                if count == 1: continue
                subdf = df[(df[c_col_names] == uniq).all(axis=1)]
                if np.unique(subdf["a"]).shape[0] == 1: continue
                e_y_a1 = subdf[subdf["a"] == 1]["y"].mean()
                e_y_a0 = subdf[subdf["a"] == 0]["y"].mean()
                total += count * (e_y_a1 - e_y_a0)
                denominator += count

            if denominator == 0:
                results.append(np.nan)
            else:
                results.append(total / denominator)

    if ground_truth is not None:
        results = [np.abs(r - ground_truth) for r in results]
    err = ""
    prec_mean, prec_std = prec
    if repeats > 1 and prec_std >= 0:
        err = f" ± {np.std(results):.{prec_std}f}"
    return f"{np.mean(results):.{prec_mean}f}{err}"

In [6]:
default_kwargs = dict(
  n=1000,
  c_dim=1,
  dice_size=6,
  repeats=10,
  power=1,
  prec=(2, 2),
  ground_truth=0.5,
)

In [7]:
experiment(n=100, c_dim=2, dice_size=3, power=3, repeats=10)
observed(n=10, c_dim=2, power=3, dice_size=2)

,a,y,c1,c2
0,1,2,2,1
1,1,2,2,1
2,1,37,1,2
3,1,1,2,1
4,1,11,1,1
5,1,2,2,1
6,1,12,1,1
7,1,2,2,1
8,0,14,2,2
9,1,34,1,2


In [13]:
kwargs = default_kwargs.copy()

for est in ["ols",  "count"]:
    kwargs["estimator"] = est
    print(est, end=" ")
    print(experiment(**kwargs))

ols 0.11 ± 0.07
count 0.10 ± 0.07


In [9]:
kwargs = default_kwargs.copy()
kwargs["power"] = 3

for est in ["ols", "count"]:
    kwargs["estimator"] = est
    print(est, end=" ")
    print(experiment(**kwargs))

ols 30.89 ± 5.68
count 1.78 ± 1.30


In [10]:
kwargs = default_kwargs.copy()
prec = (1, 1)
kwargs["prec"] = prec

powers = [1, 2, 3, 4]
n_values = [100, 1000, 10000]
c_dims = [1, 2, 4, 8]

col_header_width = int(np.log10(max(n_values))) + 3
cell_width = 6 + prec[0]
if prec[1] >= 0:
    cell_width += prec[1] + 5
    
for power in powers:
    for est in ["ols", "count"]:
    
        print(f"Power: {power}; Estimator: {est}")
        header = [f"{'c_dim':{col_header_width}}"]
        header += [f"{c:^{cell_width}}" for c in c_dims]
        print(" ".join(header))

        for n in n_values:
            runtime = time.time()
            row = [f"n={n:<{col_header_width-2}d}: "]
            for c_dim in c_dims:
                kwargs.update(dict(
                    n=n,
                    c_dim=c_dim,
                    estimator=est,
                    power=power,
                ))
                result = experiment(**kwargs)
                row.append(f"{result:{cell_width}s}")
            row.append(f"     in {time.time() - runtime:.1f}s")
            print(" ".join(row))

        print()

Power: 1; Estimator: ols
c_dim         1             2             4             8      
n=100  :  0.3 ± 0.2     0.4 ± 0.3     0.6 ± 0.4     0.5 ± 0.5          in 0.1s
n=1000 :  0.1 ± 0.1     0.2 ± 0.1     0.1 ± 0.1     0.2 ± 0.2          in 0.1s
n=10000:  0.1 ± 0.0     0.0 ± 0.0     0.0 ± 0.0     0.1 ± 0.0          in 0.4s

Power: 1; Estimator: count
c_dim         1             2             4             8      
n=100  :  0.2 ± 0.1     0.4 ± 0.3     nan ± nan     nan ± nan          in 0.3s
n=1000 :  0.1 ± 0.1     0.2 ± 0.1     0.3 ± 0.1     nan ± nan          in 1.0s
n=10000:  0.0 ± 0.0     0.1 ± 0.0     0.0 ± 0.0     1.5 ± 1.6          in 6.8s

Power: 2; Estimator: ols
c_dim         1             2             4             8      
n=100  :  1.5 ± 0.9     2.3 ± 1.4     3.1 ± 1.6     4.7 ± 2.0          in 0.1s
n=1000 :  1.7 ± 0.3     0.6 ± 0.3     0.8 ± 0.4     0.9 ± 0.8          in 0.2s
n=10000:  1.8 ± 0.1     0.2 ± 0.1     0.2 ± 0.2     0.4 ± 0.2          in 0.3s

Power: 2; Estimat

Comparisons

### Power: 1; Estimator: ols
```
c_dim          4              8      
n=100  :   0.6 ± 0.4      0.5 ± 0.5       in 0.2s
n=1000 :   0.1 ± 0.1      0.2 ± 0.2       in 0.3s
n=10000:   0.0 ± 0.0      0.1 ± 0.0       in 1.2s
```
### Power: 2; Estimator: ols
```
c_dim          2              4      
n=100  :   2.3 ± 1.4      3.1 ± 1.6       in 0.7s
n=1000 :   0.6 ± 0.3      0.8 ± 0.4       in 0.8s
n=10000:   0.2 ± 0.1      0.2 ± 0.2       in 1.5s
```
### Power: 4; Estimator: ols
```
c_dim           1             2         
n=100  : 260.5 ± 161.8  210.5 ± 188.5     in 0.3s
n=1000 : 336.4 ± 59.9    58.1 ± 27.0      in 0.4s
n=10000: 318.7 ± 20.4    20.4 ± 12.5      in 1.0s
```
### Power: 1; Estimator: count
```
c_dim          4             8      
n=100  :   nan ± nan     nan ± nan        in 0.5s
n=1000 :   0.3 ± 0.1     nan ± nan        in 2.0s
n=10000:   0.0 ± 0.0     1.5 ± 1.6        in 16.9s
```
### Power: 2; Estimator: count
```
c_dim          2             4     
n=100  :   2.2 ± 1.9     nan ± nan        in 0.6s
n=1000 :   0.4 ± 0.4     1.5 ± 0.9        in 3.1s
n=10000:   0.2 ± 0.1     0.1 ± 0.1        in 14.9s
```
### Power: 4; Estimator: count
```
c_dim          1             2       
n=100  :  10.6 ± 4.6    15.5 ± 12.0       in 0.4s
n=1000 :   3.4 ± 2.4     4.1 ± 3.9        in 3.5s
n=10000:   1.3 ± 1.1     1.0 ± 0.5        in 13.4s
```